In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd

In [2]:
import json
from pprint import pprint

with open('GSWvsCLE.json') as data_file:    
    data = json.load(data_file)

In [3]:
# read in the court png file
court = plt.imread("fullcourt.png")

headers = ["team_id", "player_id", "x_loc", "y_loc", 
           "radius"]

def playerDFtoList(frame):
    xList = frame['x_loc'].tolist()
    yList = frame['y_loc'].tolist()
    bothList = []
    for i in range(len(xList)):
        bothList.append([ xList[i], yList[i] ])
    return np.array(bothList)

In [64]:
df.head()

,team_id,player_id,x_loc,y_loc,radius
0,-1,-1,28.64068,45.40327,3.54135
1,1610612739,2544,6.44329,22.26184,0.00000
2,1610612739,2747,17.45534,4.23261,0.00000
3,1610612739,201567,7.51449,16.49044,0.00000
4,1610612739,202681,27.87412,45.48980,0.00000


In [69]:
# Get moments from the zeroth event
player_moments = []
# moments = data['events'][0]['moments']
for event in data['events']:
    for moment in event['moments']:
        for player in moment[5]:
            player_moments.append(player)

# for moment in moments:
#     for player in moment[5]:
#         player_moments.append(player)

df = pd.DataFrame(player_moments, columns=headers)

# Get specific player's movements
ball = df[df.player_id==-1]

steph = df[df.player_id==201939]
klay = df[df.player_id==202691]
bogut = df[df.player_id==101106]
green = df[df.player_id==203110]
barnes = df[df.player_id==203084]

lebron = df[df.player_id==2544]
smith = df[df.player_id==2747]
love = df[df.player_id==201567]
irving = df[df.player_id==202681]
mozgov = df[df.player_id==202389]

# convert them to numpy arrays
ballA = playerDFtoList(ball)

stephA = playerDFtoList(steph)
klayA = playerDFtoList(klay)
bogutA = playerDFtoList(bogut)
greenA = playerDFtoList(green)
barnesA = playerDFtoList(barnes)

lebronA = playerDFtoList(lebron)
smithA = playerDFtoList(smith)
loveA = playerDFtoList(love)
irvingA = playerDFtoList(irving)
mozgovA = playerDFtoList(mozgov)

#Get ball radiuses
radii = ball['radius'].tolist()

In [77]:
class AnimatedScatter(object):
    """An animated scatter plot using matplotlib.animations.FuncAnimation."""
    def __init__(self, playerLists, ballRadii):
        self.numpoints = len(playerLists)
        print 'numpoints:',self.numpoints
        self.stream = self.data_stream()

        self.ballRadii = ballRadii
        
        locations = []
        for i in range(len(playerLists[0])):
            # For each moment
            locations.append([])
            for player in playerLists:
                try:
                    locations[i].append(player[i])
                except IndexError:
                    break
        self.locations = locations
    
        if self.numpoints == 11:
            self.colorArray = np.array([0,1,1,1,1,1,2,2,2,2,2])
        else:
            self.colorArray = np.array([0] + [2]*(self.numpoints-1))
        # Setup the figure and axes...
        self.fig, self.ax = plt.subplots()
        # Then setup FuncAnimation.
        self.ani = animation.FuncAnimation(self.fig, self.update, interval=25, 
                                           init_func=self.setup_plot, blit=True)

    def setup_plot(self):
        """Initial drawing of the scatter plot."""
        pos, c, i,r = next(self.stream)
        self.scat = self.ax.scatter(2, 40, animated=True)
        self.ax.axis([0, 94, 50, 0])
#         self.ax.legend(['Event'])
        
#         self.eventnumber = plt.annotate("Event: 0", xy =(20, 30), zorder=1)

        # For FuncAnimation's sake, we need to return the artist we'll be using
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def data_stream(self):
        """Generate a random walk (brownian motion). Data is scaled to produce
        a soft "flickering" effect."""
        data = [[[],[],[]],[],[],[]]
        data[1] = self.colorArray
        
        i = 0
        while True:
            data[0] = self.locations[i]
            data[2] = i # this event number
            data[3] = self.ballRadii[i]
            i += 1
            yield data

    def update(self, i):
        """Update the scatter plot."""
        data = next(self.stream)

        # Set x and y data...
        self.scat.set_offsets(data[0])
        # Set sizes...
        self.scat._sizes = np.array([data[3]*4,120,120,120,120,120,120,120,120,120,120])
        # Set colors..
        self.scat.set_array(data[1])
        
        self.scat.set_label("Event man")
        
#         self.eventnumber.remove()
#         self.eventnumber = plt.annotate("Event: "+str(data[2]), xy =(20, 30), zorder=1)
#         draw()
        
#         self.scat.axes.legend(['poop'])

        # We need to return the updated artist for FuncAnimation to draw..
        # Note that it expects a sequence of artists, thus the trailing comma.
        return self.scat,

    def show(self):
        plt.imshow(court, zorder=0, extent=[0,94,50,0])
        plt.show()

In [ ]:
a = AnimatedScatter([ballA, stephA, klayA, bogutA, greenA, barnesA, lebronA, smithA, loveA, irvingA, mozgovA],radii)
# a = AnimatedScatter([ballA, irvingA, stephA])
# a = AnimatedScatter([irvingA])
a.show()

In [ ]:
just ball:
[
    [[28.64068], [45.403269999999999]],
    [[28.52319], [45.279890000000002]],
    [[28.38205], [45.163530000000002]]
]


with both:
[
    [[28.64068, 27.874120000000001], [45.403269999999999, 45.489800000000002]],
    [[28.52319, 27.793469999999999], [45.279890000000002, 45.431060000000002]],
    [[28.38205, 27.717929999999999], [45.163530000000002, 45.364379999999997]]
]